In [1]:
import numpy as np
import pandas as pd
import cv2
from datetime import datetime
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2

In [2]:
MARGIN = 10  # pixels
FONT_SIZE = 1
FONT_THICKNESS = 1
HANDEDNESS_TEXT_COLOR = (88, 205, 54) # vibrant green

def draw_landmarks_on_image(rgb_image, detection_result):
  hand_landmarks_list = detection_result.hand_landmarks
  handedness_list = detection_result.handedness
  annotated_image = np.copy(rgb_image)

  # Loop through the detected hands to visualize.
  for idx in range(len(hand_landmarks_list)):
    hand_landmarks = hand_landmarks_list[idx]
    handedness = handedness_list[idx]

    # Draw the hand landmarks.
    hand_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
    hand_landmarks_proto.landmark.extend([
      landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in hand_landmarks
    ])
    solutions.drawing_utils.draw_landmarks(
      annotated_image,
      hand_landmarks_proto,
      solutions.hands.HAND_CONNECTIONS,
      solutions.drawing_styles.get_default_hand_landmarks_style(),
      solutions.drawing_styles.get_default_hand_connections_style())

    # Get the top left corner of the detected hand's bounding box.
    height, width, _ = annotated_image.shape
    x_coordinates = [landmark.x for landmark in hand_landmarks]
    y_coordinates = [landmark.y for landmark in hand_landmarks]
    text_x = int(min(x_coordinates) * width)
    text_y = int(min(y_coordinates) * height) - MARGIN

    # Draw handedness (left or right hand) on the image.
    cv2.putText(annotated_image, f"{handedness[0].category_name} [{handedness[0].index}]",
                (text_x, text_y), cv2.FONT_HERSHEY_DUPLEX,
                FONT_SIZE, HANDEDNESS_TEXT_COLOR, FONT_THICKNESS, cv2.LINE_AA)

  return annotated_image

In [3]:
# Use OpenCV’s VideoCapture to start capturing from the webcam.
webcam = cv2.VideoCapture(0)

if not webcam.isOpened():
    print("Could not open webcam")
    exit()

In [4]:
model_path = './models/hand_landmarker.task'

BaseOptions = mp.tasks.BaseOptions
HandLandmarker = mp.tasks.vision.HandLandmarker
HandLandmarkerOptions = mp.tasks.vision.HandLandmarkerOptions
HandLandmarkerResult = mp.tasks.vision.HandLandmarkerResult
VisionRunningMode = mp.tasks.vision.RunningMode

class storage:
    def __init__(self):
        self.image = np.zeros((480, 640), dtype=np.uint8)

        self.size = 100
        self.hand_data = [{'x':np.zeros(shape=self.size), 'y':np.zeros(shape=self.size), 'z':np.zeros(shape=self.size)},{'x':np.zeros(shape=self.size), 'y':np.zeros(shape=self.size), 'z':np.zeros(shape=self.size)},{},{},{}]
        self.index = 0

    def get_annotated_image(self):
        return self.image
    
    def save_annotated_image(self, img):
        self.image = img

    def print_data(self):
        print(self.hand_data)        

    def save_data(self, i, x, y, z):
        self.hand_data[i]['x'][self.index] = x
        self.hand_data[i]['y'][self.index] = y
        self.hand_data[i]['z'][self.index] = z

        if(self.index < self.size-1):
            self.index += 1
        else:
            self.index = 0


# 이미지 저장하는 클래스
image_storage = storage()
# 데이터 저장하는 클래스
data_storage = storage()


In [5]:

# Create a hand landmarker instance with the live stream mode:
def print_result(result: HandLandmarkerResult, output_image: mp.Image, timestamp_ms: int):
    # print('hand landmarker result: {}'.format(result))
    
    hand_landmarks_list = result.hand_landmarks
    handedness_list = result.handedness
    for i in range(len(hand_landmarks_list)):
        #print(hand_landmarks_list[i][0].x)
        hll = hand_landmarks_list[i][0]
        data_storage.save_data(i, hll.x, hll.y, hll.z)
        #hand_data[i] = hand_landmarks_list[i]
        
    detection_result = result
    mp_image = output_image
    annotated_image = draw_landmarks_on_image(mp_image.numpy_view(), detection_result)
    image_storage.save_annotated_image(annotated_image)

    
# Create a hand landmarker instance with the image mode:
options = HandLandmarkerOptions(
    base_options=BaseOptions(model_asset_path=model_path),
    running_mode=VisionRunningMode.LIVE_STREAM,
    result_callback=print_result,
    num_hands=5,
    min_hand_detection_confidence = 0.5,
    min_hand_presence_confidence = 0.5,
    min_tracking_confidence = 0.5)


In [6]:
with HandLandmarker.create_from_options(options) as landmarker:
    # The landmarker is initialized. Use it here.

    # Create a loop to read the latest frame from the camera using VideoCapture#read()
    while webcam.isOpened():
        status, frame = webcam.read()
        frame_timestamp_ms = int(webcam.get(cv2.CAP_PROP_POS_MSEC))

        # Convert the frame received from OpenCV to a MediaPipe’s Image object.
        mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame)
        
        # Send live image data to perform hand landmarks detection.
        # The results are accessible via the `result_callback` provided in
        # the `HandLandmarkerOptions` object.
        # The hand landmarker must be created with the live stream mode.
        landmarker.detect_async(mp_image, frame_timestamp_ms)
        
        
        if status:
            cv2.imshow("test", image_storage.get_annotated_image())
            #cv2.imshow("test", annotated_image)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break 

    cv2.destroyAllWindows()

    data_storage.print_data()

[{'x': array([ 0.02014948,  0.01693565,  0.03012998,  0.01704004,  0.01917774,
        0.02516751,  0.01613072,  0.0303416 ,  0.01466306,  0.02297798,
        0.0216208 ,  0.01478827,  0.00803299,  0.00115822, -0.00029737,
       -0.00562461,  0.06105779, -0.01376158, -0.01393087,  0.05896315,
       -0.01965718, -0.02365541, -0.02454628, -0.02874233, -0.02999294,
        0.05963606, -0.02111222,  0.0555456 , -0.0192465 , -0.01947086,
        0.04985751, -0.02181248,  0.03736691, -0.01701735, -0.01667486,
       -0.02035047, -0.01739353, -0.02287075, -0.02457167, -0.02336608,
       -0.01952685, -0.02247067, -0.01809701, -0.00933675, -0.01472926,
       -0.00872247, -0.00720814, -0.00779161, -0.00368969,  0.00154391,
       -0.00203075,  0.01521604,  0.01045176,  0.02008013,  0.01285002,
        0.01755749,  0.01995514,  0.02011482,  0.01911526,  0.01734737,
        0.02133583,  0.020943  ,  0.01681316,  0.02692547,  0.01680639,
        0.02946543,  0.02030079,  0.02435408,  0.02585813

In [7]:
# 웹캠의 연결을 끊어준다.
# webcam.release() 